In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, CuDNNLSTM
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score,roc_auc_score, precision_score, recall_score
import Models
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from importlib import reload 
import numpy as np
import math
import re
reload(Models)

Using TensorFlow backend.


<module 'Models' from '/home/zhanglab/Desktop/Hyunmin/protein/Models.py'>

In [2]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):
        y_pred = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        y_true = self.validation_data[1]
        
        tp = np.sum(np.multiply(y_pred, y_true), axis=1)
        pred_p = np.sum(y_pred, axis=1)
        true_p = np.sum(y_true, axis=1)
        
        precision = np.mean(np.nan_to_num(tp/pred_p))
        recall = np.mean(tp/true_p)
        f1 = np.mean(2*np.multiply(precision,recall)/(precision+recall))

        self.val_f1s.append(f1)
        self.val_recalls.append(recall)
        self.val_precisions.append(precision)
        f1 = f1_score(y_true,y_pred,average='weighted')
        precision = precision_score(y_true,y_pred,average='weighted')
        recall = recall_score(y_true,y_pred,average='weighted')
        try:
            roc_auc = roc_auc_score(y_true,y_pred,average='weighted')
        except:
            roc_auc = 0
        print("— val_f1: %f — val_precision: %f — val_recall %f" %(f1, precision, recall))
        return
     
metrics = Metrics()
def evaluate(y_pred,y_true):
    f1 = f1_score(y_true,y_pred,average='weighted')
    precision = precision_score(y_true,y_pred,average='weighted')
    recall = recall_score(y_true,y_pred,average='weighted')
    try:
        roc_auc = roc_auc_score(y_true,y_pred,average='weighted')
    except:
        roc_auc = 0
    print("— test_f1: %f — test_precision: %f — test_recall %f" %(f1, precision, recall))
    return f1

In [3]:
os.chdir('.')
X = np.load('aa_seq_onehot.npy') # one hot encoding of protein seq, flattened
Y = np.load('solvents_onehot.npy') # one hot encoding of solvents, see order above


X = X.reshape(X.shape[0],X.shape[1],1)
Y = Y

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1,random_state=42)


counts = np.sum(Y,axis=0)
max_num = np.max(counts)
class_weight = {}
for i in range(Y.shape[1]):
    class_weight[i] = max_num/counts[i]

input_size = 40000
batch_size = 64

In [4]:
reload(Models)
input_size = 40000
batch_size = 64
epochs = 40
from keras import optimizers
opt = optimizers.Adam(0.0001)
model = Models.Convolutional_Feedforward((input_size,1),192)
model.compile(loss='binary_crossentropy',optimizer=opt,
metrics=['accuracy']) 
#es = EarlyStopping(monitor='acc', mode='min',patience=50, verbose=1)

In [5]:
history = model.fit(X_train, Y_train,
                    class_weight=class_weight,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,validation_split=0.1,
                    callbacks=[metrics],shuffle=True)
score = model.evaluate(X_test, Y_test,
                       batch_size=batch_size, verbose=1)

Train on 58043 samples, validate on 6450 samples
Epoch 1/40
58043/58043 [==============================] - 121s 2ms/step - loss: 0.2001 - acc: 0.9566 - val_loss: 0.1452 - val_acc: 0.9725


/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/

— val_f1: 0.132331 — val_precision: 0.124585 — val_recall 0.141104
Epoch 2/40
58043/58043 [==============================] - 113s 2ms/step - loss: 0.1364 - acc: 0.9707 - val_loss: 0.1269 - val_acc: 0.9701


/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/

— val_f1: 0.071542 — val_precision: 0.482847 — val_recall 0.046233
Epoch 3/40
58043/58043 [==============================] - 113s 2ms/step - loss: 0.1225 - acc: 0.9720 - val_loss: 0.1288 - val_acc: 0.9730


...

— val_f1: 0.132709 — val_precision: 0.123460 — val_recall 0.143455
Epoch 39/40
58043/58043 [==============================] - 112s 2ms/step - loss: 0.1268 - acc: 0.9725 - val_loss: 0.1275 - val_acc: 0.9725


/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average,

— val_f1: 0.132709 — val_precision: 0.123460 — val_recall 0.143455
Epoch 40/40
58043/58043 [==============================] - 112s 2ms/step - loss: 0.1269 - acc: 0.9725 - val_loss: 0.1275 - val_acc: 0.9725


/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average,

— val_f1: 0.132709 — val_precision: 0.123460 — val_recall 0.143455
7166/7166 [==============================] - 2s 281us/step


In [18]:
from keras.models import Model
models = Model(inputs=model.inputs, outputs=model.layers[1].output)

In [19]:
feature_maps = models.predict(X_val)

In [20]:
from matplotlib import pyplot

In [ ]:
ix = 1
square=8
for _ in range(square):
	for _ in range(square):
		# specify subplot and turn of axis
		ax = pyplot.subplot(square, square, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		pyplot.plot(feature_maps[0,:,ix-1])
		ix += 1
# show the figure
pyplot.show()

In [9]:
evaluate(model.predict(X_test).round(),Y_test)

/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/zhanglab/miniconda2/envs/concise/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average,

— test_f1: 0.462745 — test_precision: 0.517564 — test_recall 0.443241


0.46274481151694763

In [101]:
evaluate(model.evaluate(X_test), Y_test)

IndexError: list index out of range

In [49]:
evaluate(model.predict(X_train).round(), Y_train)

KeyboardInterrupt: 

In [69]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
precision = dict()
recall = dict()
average_precision = dict()
for i in range(192):
    precision[i], recall[i], _ = precision_recall_curve(model.predict(X_test).round()[:,i], Y_test[:,i])
    average_precision[i] = average_precision_score(model.predict(X_test).round()[:,i], Y_test[:,i])

/home/zhanglab/anaconda3/envs/rna/lib/python3.6/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide


KeyboardInterrupt: 

In [14]:
for i in X_test:
    for j in X_train:
        

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[ True]
 [False]
 [False]
 ...
 [False]

 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[ True]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[ True]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[ True]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ..

 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[ True]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[ True]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 ..

In [12]:
X_test.shape

(7166, 40000, 1)